# Laboratorium 3 - segmentacje

## Przygotowanie

 * pobierz i wypakuj dataset: https://data.world/socialmediadata/beeradvocate
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab3`
 * zainstaluj potrzebne biblioteki:
 `pip install gensim==3.8.3 scikit-learn==1.3.2 wordcloud==1.8.1`

## Część 1. - przygotowanie danych

In [1]:
# importujemy potrzebne pakiety

from collections import Counter, defaultdict
import codecs
import csv

from gensim.corpora import Dictionary
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, silhouette_samples
from wordcloud import WordCloud

In [2]:
PATH = 'socialmediadata-beeradvocate/data/beer_reviews.csv'
SCORE_THRESHOLD = 4 # recenzje z co najmniej taka ocena wezmiemy pod uwage
VECTOR_SIZE = 20 # jak dlugie powinny byc wektory osadzen uzytkownikow
SEGMENTS_COUNT = 10 # na ile segmentow chcemy podzielic populacje uzytkownikow

In [4]:
# wczytujemy dane

def parse_headers(column_names):
    beer_id_column =  column_names.index('beer_beerid')
    beer_style_column = column_names.index('beer_style')
    user_id_column =  column_names.index('review_profilename')
    score_column =  column_names.index('review_overall')
    return beer_id_column, beer_style_column, user_id_column, score_column


def parse_review(line, beer_id_column, beer_style_column, user_id_column, score_column):
    return line[beer_id_column], line[beer_style_column], line[user_id_column], float(line[score_column])

def read_and_parse_reviews(path, score_threshold):
    with codecs.open(PATH, 'r', 'UTF-8') as datafile:
        datareader = csv.reader(datafile)
        beer_id_column, beer_style_column, user_id_column, score_column = parse_headers(next(datareader))
    
        users_favourite_beers = defaultdict(list)
        for review in datareader:
            beer_id, _, user_id, score = parse_review(review, beer_id_column, beer_style_column, user_id_column, score_column)
            if score >= score_threshold:
                users_favourite_beers[user_id].append(beer_id)

    return users_favourite_beers

def get_beer_id_to_style_mapping(path):
    with codecs.open(PATH, 'r', 'UTF-8') as datafile:
        datareader = csv.reader(datafile)
        beer_id_column, beer_style_column, user_id_column, score_column = parse_headers(next(datareader))
        beer_styles = {}
        for review in datareader:
            beer_id, beer_style, _, _ = parse_review(review, beer_id_column, beer_style_column, user_id_column, score_column)
            beer_styles[beer_id] = beer_style
        return beer_styles

# otrzymujemy slownik - mapowanie z user_id na liste ulubionych beer_ids
users_favourite_beers = read_and_parse_reviews(PATH, SCORE_THRESHOLD)

# dodatkowo przygotujmy sobie slownik mapujacy id piwa na nazwe stylu
beer_styles = get_beer_id_to_style_mapping(PATH)

## Część 2. - osadzenia użytkowników

In [5]:
# trenujemy model osadzajacy piwa i uzytkownikow w przestrzeni

model = Word2Vec(sentences=users_favourite_beers.values(), size=VECTOR_SIZE, window=5, min_count=1, workers=4)

In [6]:
# na podstawie modelu obliczamy wektorowa reprezentacje uzytkownikow

def get_mean_vector_for_user(user_favourite_beers, model, vector_size):
    # inicjalizujemy wektor zerami
    v = np.zeros(vector_size)
    
    # iterujemy po liscie user_favourite_beers, odczytujemy z modelu wektor reprezentujacy kazde piwo i dodajemy do wektora
    #   uzyj: model.wv[beer_id]
    # ...
    for beer_id in user_favourite_beers:
        v+=model.wv[beer_id]
    
    # normalizujemy wektor - dzielimy kazda wspolrzedna przez liczbe piw
    # ...
    v /= len(user_favourite_beers)
    return v

def get_mean_user_vectors(users_favourite_beers, model, vector_size):
    # korzystajac z powyzszej funkcji, tworzymy slownik {user_id -> vector}
    mean_users_vectors = {}
    for user in users_favourite_beers:
        mean_users_vectors[user] = get_mean_vector_for_user(users_favourite_beers[user],model,vector_size)
    return mean_users_vectors

user_vectors = get_mean_user_vectors(users_favourite_beers, model, VECTOR_SIZE)
user_vectors

{'johnmichaelsen': array([ 0.65383548,  0.28704705, -0.48296683,  0.65534185, -0.36294626,
        -0.76046982,  0.33550055,  0.47196825, -0.37341455, -0.54503397,
        -0.23006729,  0.58621394, -0.64205284,  0.73040251,  0.56231995,
        -0.11745093,  1.26412096, -0.69316942,  0.76088647, -0.7820165 ]),
 'LordAdmNelson': array([ 0.56615941,  0.73445429, -0.80007715,  0.96146302, -0.69823398,
        -1.10322188,  0.71603007,  0.94715001, -0.35654268, -0.45634429,
        -0.41808666,  0.72993375, -0.96916962,  1.21505971,  0.61139171,
        -0.01940001,  2.4222837 , -1.74337612,  1.08761309, -1.54732222]),
 'augustgarage': array([ 0.83456216,  0.35122078, -0.71352561,  0.75369266, -0.34782236,
        -1.02402807,  0.3799281 ,  0.58418857, -0.41680542, -0.68061579,
        -0.2834853 ,  0.66439317, -0.69593132,  0.96022976,  0.71799487,
        -0.15841257,  1.62953526, -0.85440892,  0.96291975, -1.07170301]),
 'MadeInOregon': array([ 0.66426269,  0.6524515 , -0.28854065,  0.8

## Część 3. - klasteryzacja

In [7]:
# klastrujemy uzytkownikow - mozemy uzyc wielu algorytmow, np k-means, agglomerative, BIRCH, ...

def get_users_segmentation(user_vectors, vector_size, segments_count, clustering_algorithm):
    # rozkladamy slownik user_vectors na liste uzytkownikow i liste wektorow
    #  - wazne jest zachowanie tej samej kolejnosci w obu listach
    users = user_vectors.keys()
    vectors = user_vectors.values()
    
    # zamieniamy liste wektorow w macierz
    users_array = np.stack(vectors, axis=0)
    # zaimplementuj wsparcie dla co najmniej jednego algorytmu wiecej
    if clustering_algorithm == 'agglomerative':
        clustering = AgglomerativeClustering(n_clusters=segments_count).fit_predict(users_array)
    elif clustering_algorithm == 'k_means':
        clustering = KMeans(n_clusters=segments_count).fit_predict(users_array)

        # clustering to lista przypisanych klastrow - i-ty element to klaster, do ktorego nalezy i-ty wektor
    
    segmentation = {}
    # jesli nie pomieszalismy kolejnosci w listach, to mozemy odzyskac mapping user_id -> cluster
    # ...
    for i,user_id in enumerate(users):
        segmentation[user_id] = clustering[i]
    return segmentation

segmentation = get_users_segmentation(user_vectors, VECTOR_SIZE, SEGMENTS_COUNT, 'agglomerative')
segmentation

c:\Users\mateu\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3378: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


{'johnmichaelsen': 5,
 'LordAdmNelson': 7,
 'augustgarage': 7,
 'MadeInOregon': 5,
 'rawthar': 7,
 'Halcyondays': 5,
 'beerguy101': 5,
 'UCLABrewN84': 7,
 'zaphodchak': 7,
 'Tilley4': 7,
 'mikedrinksbeer2': 7,
 'dbmernin83': 7,
 'titosupertramp': 7,
 'optimator13': 7,
 'Blakaeris': 7,
 'bashiba': 7,
 'Klym': 7,
 'flexabull': 7,
 'Bung': 7,
 'oberon': 5,
 't420o': 7,
 'DarthKostrizer': 7,
 'MrVonzipper': 7,
 'TheWhiskerReview': 7,
 'Jason': 5,
 'DoubleJ': 5,
 'Deuane': 5,
 'Mattybeerguy': 5,
 'micromaniac129': 7,
 'northyorksammy': 5,
 'vette2006c5r': 7,
 'EvanEngelstad': 5,
 'djeucalyptus': 7,
 'ccrida': 5,
 'brewandbbq': 7,
 'hopsbreath': 7,
 'Slatetank': 7,
 'Proteus93': 7,
 'SShelly': 7,
 'zoso1967': 7,
 'msubulldog25': 7,
 'NJpadreFan': 7,
 'd0ggnate': 7,
 'Cyberkedi': 5,
 'babyhobbes': 7,
 'mdagnew': 5,
 'mikesgroove': 5,
 'hdofu': 7,
 'GratefulBeerGuy': 7,
 'PDXHops': 7,
 'IMincedTheKing': 7,
 'Goliath': 7,
 'zeff80': 5,
 'akorsak': 5,
 'wchesak': 7,
 'bfields4': 7,
 'dyan': 7,
 

In [8]:
# obliczamy licznosci segmentow

def get_segment_sizes(segmentation):
    sizes = {}
    for segment in segmentation.values():
        sizes[segment]=0
    for segment in segmentation.values():
        sizes[segment]+=1

    return sizes

get_segment_sizes(segmentation)

{5: 4865,
 7: 12223,
 3: 3058,
 4: 1633,
 2: 2611,
 0: 1596,
 1: 2087,
 6: 984,
 9: 674,
 8: 555}

In [9]:
# obliczamy metryke jakosci segmentacji
# metryka przyjmuje wartosci [-1, 1] - im wiecej, tym lepiej
users, user_vectors_list = zip(*user_vectors.items())
segments_list = [segmentation[u] for u in users]

mean_silhouette = silhouette_score(user_vectors_list, segments_list, metric='euclidean')
per_sample_silhouettes = silhouette_samples(user_vectors_list, segments_list, metric='euclidean')

In [10]:
# rysujemy wykres
def plot_histogram(values, mean_value):
    plt.hist(values, color='c', edgecolor='k', alpha=0.65)
    plt.axvline(mean_value, linestyle='dashed', linewidth=1)
    _, plot_height = plt.ylim()
    plt.text(0, plot_height*1.05, 'Mean: {:.3f}'.format(mean_value))
    plt.show()

plot_histogram(per_sample_silhouettes, mean_silhouette)

## Część 4. - opisy segmentów

In [11]:
# obliczamy popularnosc styli w segmentach oraz w calej populacji
#  - czyli jak czesto kazdy ze styli wystepowal

def get_per_segment_styles_popularity(users_favourite_beers, beer_styles, segmentation, segments_count):
    beer_styles_per_segment = {i: defaultdict(lambda: 0) for i in range(segments_count)}
    # do tego slownika bedziemy sie odwolywac np. tak: beer_styles_per_segment[segment][style]
    
    for user, beers in users_favourite_beers.items():
        for beer in beers:
            beer_styles_per_segment[segmentation[user]][beer_styles[beer]]+=1
    
    return beer_styles_per_segment

def get_total_styles_popularity(beer_styles_per_segment):
    total_popularity = defaultdict(lambda: 0)
    for segment in beer_styles_per_segment.keys():
        for style in beer_styles_per_segment[segment]:
            total_popularity[style] += beer_styles_per_segment[segment][style]
    return total_popularity

per_segment_styles_popularity = get_per_segment_styles_popularity(users_favourite_beers, beer_styles, segmentation, SEGMENTS_COUNT)
print(per_segment_styles_popularity)
total_styles_popularity = get_total_styles_popularity(per_segment_styles_popularity)
print(total_styles_popularity)

{0: defaultdict(<function get_per_segment_styles_popularity.<locals>.<dictcomp>.<lambda> at 0x000001E3AEBC0E50>, {'Light Lager': 95, 'American Porter': 142, 'Czech Pilsener': 85, 'English Brown Ale': 93, 'Hefeweizen': 69, 'Fruit / Vegetable Beer': 113, 'American Pale Ale (APA)': 180, 'Belgian Strong Dark Ale': 132, 'Dubbel': 85, 'California Common / Steam Beer': 18, 'Vienna Lager': 119, 'Irish Red Ale': 62, 'Märzen / Oktoberfest': 117, 'Winter Warmer': 79, 'Schwarzbier': 102, 'English Pale Ale': 82, 'American Pale Wheat Ale': 130, 'Milk / Sweet Stout': 70, 'Scottish Ale': 21, 'American Strong Ale': 21, 'Dortmunder / Export Lager': 15, 'English India Pale Ale (IPA)': 18, 'Witbier': 235, 'Extra Special / Strong Bitter (ESB)': 25, 'American Pale Lager': 25, 'American Amber / Red Ale': 118, 'Munich Helles Lager': 17, 'Bock': 131, 'Munich Dunkel Lager': 17, 'Pumpkin Ale': 46, 'Oatmeal Stout': 88, 'Euro Pale Lager': 33, 'American Adjunct Lager': 229, 'Black & Tan': 7, 'American Amber / Red L

In [12]:
# przygotujmy dwa opisy segmentow - top N najpopularniejszych stylow w kazdym segmencie
#   oraz top N unikalnych - to samo, ale popularnosc w segmencie dzielimy przez laczna popularnosc
#   dla potrzeb wizualizacji w nastepnym kroku, zwroc slownik {segment_id -> {style_id -> frequency}}

def most_popular_styles_per_segment(per_segment_styles_popularity, N):
    mpsps = {}
    for segment in per_segment_styles_popularity.keys():
        styles = sorted(per_segment_styles_popularity[segment].items(), key = lambda x:x[1], reverse=True)
        mpsps[segment] = dict(styles[:N])
    return mpsps

def most_distinctive_styles_per_segment(total_styles_popularity, per_segment_styles_popularity, N):
    mdsps = {}
    for segment in per_segment_styles_popularity.keys():
        for style in per_segment_styles_popularity[segment].keys():
            per_segment_styles_popularity[segment][style] /= total_styles_popularity[style]

    for segment in per_segment_styles_popularity.keys():
        styles = sorted(per_segment_styles_popularity[segment].items(), key = lambda x:x[1], reverse=True)
        mdsps[segment] = dict(styles[:N])
    return mdsps

     
top_N = 50
most_popular = most_popular_styles_per_segment(per_segment_styles_popularity, top_N)
most_distinctive = most_distinctive_styles_per_segment(total_styles_popularity, per_segment_styles_popularity, top_N)

## Część 5. - wizualizacja

In [13]:
# teraz mozemy zwizualizowac nasze segmentacje za pomoca word clouds
#   zauwaz, jak roznia sie obie metody opisow segmentow oraz opisy miedzy poszczegolnymi segmentami

def visualise_styles(most_popular, most_distinctive, segment_id):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    wordcloud1 = WordCloud(random_state=42, background_color='black', colormap='Set2')
    wordcloud1.generate_from_frequencies(frequencies=most_popular[segment_id])
    ax1.imshow(wordcloud1, interpolation="bilinear")
    wordcloud2 = WordCloud(random_state=42, background_color='black', colormap='Set2')
    wordcloud2.generate_from_frequencies(frequencies=most_distinctive[segment_id])
    ax2.imshow(wordcloud2, interpolation="bilinear")
    ax1.axis("off")
    ax2.axis("off")
    plt.show()
    
visualise_styles(most_popular, most_distinctive, 3)